In [46]:
%matplotlib inline
import matplotlib.pyplot as plt

import pandas as pd

all_cards = pd.read_csv('data/card_data.csv')
print(all_cards.describe())

# 属性なしって必ずモンスターなのか?
monsters = all_cards[pd.notnull(all_cards['Attribute'])]

             Level          ATK          DEF
count  4275.000000  4275.000000  4275.000000
mean      4.449591  1455.553216  1212.318129
std       2.185045   899.165365   802.966529
min       1.000000     0.000000     0.000000
25%       3.000000   800.000000   600.000000
50%       4.000000  1500.000000  1200.000000
75%       6.000000  2100.000000  1800.000000
max      12.000000  5000.000000  5000.000000


In [103]:
x = monsters.drop(columns='Attribute')
t = monsters[['Attribute']]

# class MyML:
#     def __init__(self):
    
#     def convert_to_int_of(column_name):
#         kinds = create_kinds_list_of(column_name)
#         di = create_kinds_int_dictionary(kinds)
#         return t.replace({column_name: di})
    
def create_kinds_list_of(df, column_name):
    return sorted(df[column_name].unique().tolist())
    
def create_kinds_int_dictionary(kinds):
    dict = {}
    for kind in kinds:
        dict[kind] = kinds.index(kind)
        
    return dict

def convert_to_int_of(df, column_name):
    kinds = create_kinds_list_of(df, column_name)
    di = create_kinds_int_dictionary(kinds)
    return df.replace({column_name: di})

# 属性の例: ['DARK', 'DIVINE', 'EARTH', 'FIRE', 'LIGHT', 'WATER', 'WIND']
t = convert_to_int_of(t, "Attribute")


x = convert_to_int_of(x, "Type")
x = convert_to_int_of(x, "Race")
# print(x.head(5))

import sklearn.preprocessing as sp

# この変換はダメだ。単に多重度に応じて数字を割り当てているだけ。
# pd.set_option("display.max_rows", 50)
le = sp.LabelEncoder()
le.fit(x["Name"].unique())
# print(len(le.transform(x["Name"])))

# print(len(x["Name"]))
# print(len(x["Name"].unique()))
# print(x[x.duplicated(subset=["Name"], keep=False)])
x["Name"] = le.transform(x["Name"])

t = t.astype('int32')
x = x.astype('float32')

import chainer
chainer.print_runtime_info()

# データセットを分割するモジュールの読み込み
from sklearn.model_selection import train_test_split
x_train_val, x_test, t_train_val, t_test = train_test_split(x, t, test_size=0.3, random_state=0)

x_train, x_val, t_train, t_val = train_test_split(x_train_val, t_train_val, test_size=0.3, random_state=0)


import chainer.links as L
import chainer.functions as F

# 入力次元数が 6、出力次元数が 7 の全結合層
l = L.Linear(6, 7)

from chainer import Sequential

# net としてインスタンス化
n_input = 6
n_hidden = 10
n_output = 7

# ReLUって減衰が起きるんじゃなかったっけ
net = Sequential(
    L.Linear(n_input, n_hidden), F.relu,
    L.Linear(n_hidden, n_hidden), F.relu,
    L.Linear(n_hidden, n_output)
)

optimizer = chainer.optimizers.SGD(lr=0.01)

optimizer.setup(net)

# 数値コピペ
n_epoch = 30
n_batchsize = 16

import numpy as np

iteration = 0

# ログの保存用
results_train = {
    'loss': [],
    'accuracy': []
}
results_valid = {
    'loss': [],
    'accuracy': []
}


for epoch in range(n_epoch):

    # データセット並べ替えた順番を取得
    order = np.random.permutation(range(len(x_train)))

    # 各バッチ毎の目的関数の出力と分類精度の保存用
    loss_list = []
    accuracy_list = []

    for i in range(0, len(order), n_batchsize):
        # バッチを準備
        index = order[i:i+n_batchsize]
        x_train_batch = x_train[index,:]
        t_train_batch = t_train[index]

        # 予測値を出力
        y_train_batch = net(x_train_batch)

        # 目的関数を適用し、分類精度を計算
        loss_train_batch = F.softmax_cross_entropy(y_train_batch, t_train_batch)
        accuracy_train_batch = F.accuracy(y_train_batch, t_train_batch)

        loss_list.append(loss_train_batch.array)
        accuracy_list.append(accuracy_train_batch.array)

        # 勾配のリセットと勾配の計算
        net.cleargrads()
        loss_train_batch.backward()

        # パラメータの更新
        optimizer.update()

        # カウントアップ
        iteration += 1

    # 訓練データに対する目的関数の出力と分類精度を集計
    loss_train = np.mean(loss_list)
    accuracy_train = np.mean(accuracy_list)

    # 1エポック終えたら、検証データで評価
    # 検証データで予測値を出力
    with chainer.using_config('train', False), chainer.using_config('enable_backprop', False):
        y_val = net(x_val)

    # 目的関数を適用し、分類精度を計算
    loss_val = F.softmax_cross_entropy(y_val, t_val)
    accuracy_val = F.accuracy(y_val, t_val)

    # 結果の表示
    print('epoch: {}, iteration: {}, loss (train): {:.4f}, loss (valid): {:.4f}'.format(
        epoch, iteration, loss_train, loss_val.array))

    # ログを保存
    results_train['loss'] .append(loss_train)
    results_train['accuracy'] .append(accuracy_train)
    results_valid['loss'].append(loss_val.array)
    results_valid['accuracy'].append(accuracy_val.array)

Platform: Linux-4.15.0-62-generic-x86_64-with-debian-buster-sid
Chainer: 6.3.0
NumPy: 1.17.2
CuPy: Not Available
iDeep: Not Available


TypeError: '(array([ 722, 1914,  783, 1405, 1613, 1074, 1151,  170,  667,  806,  480,
         29, 1782,  813, 1937, 2071]), slice(None, None, None))' is an invalid key